In [21]:
import pandas as pd
from report_manager import project, dataset, report, analyses, plots
import networkx as nx
from networkx.readwrite import json_graph
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from scipy.stats import zscore
init_notebook_mode(connected=True)
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [2]:
my_project = project.Project(identifier='P0000001', datasets=None, report={})

In [3]:
proteomics_dataset = my_project.get_dataset('proteomics')

In [4]:
my_project.list_datasets()

dict_keys(['multiomics', 'clinical', 'proteomics'])

In [5]:
my_project.get_dataset('proteomics').list_dataframes()

['bootstrap-coupled estimation dabest',
 'correlation correlation',
 'go annotation',
 'number of modified proteins',
 'number of peptides',
 'number of proteins',
 'original',
 'pathway annotation',
 'processed',
 'protein biomarkers',
 'regulated',
 'regulation table']

In [6]:
proteomics_dataset = my_project.get_dataset('proteomics')
processed_df = proteomics_dataset.get_dataframe('processed')

In [7]:
processed_df.head()

,A2M~P01023,A30~A2MYE2,ABI3BP~Q7Z7G0,ACTB~P60709,ACTN1~P12814,ADAMTS13~Q76LX8,ADH4~P08319,AFM~P43652,AGT~P01019,AHSG~P02765,...,~Q9UL83,~Q9UL86,~Q9UL89,~Q9UL90,~Q9UL94,~S6B294,~S6BAM6,~S6BAR0,~S6BGE9,~S6C4R7
0,38.005564,28.173504,21.855967,27.090330,25.039968,24.010605,23.389032,32.775470,31.758752,33.336045,...,30.775725,27.987610,28.499335,30.777842,27.099789,25.691257,25.808454,27.727998,25.699741,29.907056
1,37.309118,27.981907,27.342062,27.461155,25.896268,24.135818,22.148706,32.667632,31.857234,33.492260,...,29.316590,27.223798,28.161442,25.822550,27.854599,26.397396,25.672685,27.825901,25.142581,28.994235
2,37.384952,28.857627,21.290405,27.929764,24.295225,24.121788,23.017163,32.798461,31.952826,33.477498,...,30.997052,27.013292,29.028400,28.612315,26.334802,25.862015,26.322769,27.174838,26.070833,29.526754
3,38.417225,28.978380,25.501910,27.152479,25.231288,24.568309,26.388112,32.772464,31.740090,33.829414,...,30.911708,27.486311,29.887553,30.217247,27.100391,26.908390,27.385909,27.731448,25.825052,29.565273
4,37.471303,28.748744,21.352688,27.537048,22.392992,24.961173,24.339540,31.436090,32.167194,33.276558,...,30.821225,28.434917,30.159666,29.399582,26.677209,26.128864,25.733994,27.593356,27.354369,29.774909


In [8]:
processed_df = processed_df.drop(['sample', 'subject'], axis=1).set_index('group').apply(zscore).reset_index()

In [9]:
proteomics_report = my_project.get_dataset('proteomics').report

In [10]:
proteomics_report.list_plots()

dict_keys(['0_date', '10~stratification_pca~pca', '11~regulation_anova~basicTable', '12~regulation_anova~volcanoplot', '13~bootstrap-coupled estimation_dabest~basicTable', '14~correlation_correlation~network', '15~correlation_correlation~heatmap', '16~interaction_network~network', '17~drug_associations~basicTable', '18~disease_associations~basicTable', '19~literature_associations_publications_abstracts~basicTable', '1~peptides~barplot', '20~literature_associations_publications_abstracts~wordcloud', '21~go_enrichment_regulation_enrichment~basicTable', '22~pathway_enrichment_regulation_enrichment~basicTable', '2~peptides~basicTable', '3~proteins~barplot', '4~proteins~basicTable', '5~modifications~facetplot', '6~modifications~basicTable', '7~ranking_ranking_with_markers~ranking', '8~ranking_ranking_with_markers~basicTable', '9~coefficient_variation_coefficient_of_variation~scatterplot_matrix', 'proteomics_pipeline~cytoscape_network'])

In [22]:
proteomics_report.visualize_plot(environment='notebook', plot_type='14~correlation_correlation~network')

Cytoscape(data={'elements': [{'data': {'degree': 2, 'radius': 2, 'color': '#1acf66', 'cluster': 0, 'id': 'APOC…

In [12]:
correlation_net_dict = proteomics_report.get_plot('14~correlation_correlation~network')[0]

In [13]:
correlation_net = json_graph.node_link_graph(correlation_net_dict['net_json'])

In [14]:
correlation_df = pd.DataFrame.from_dict(correlation_net.nodes(data=True))
correlation_df = correlation_df[0].to_frame().join(correlation_df[1].apply(pd.Series))

In [15]:
correlation_df.columns = ['identifier', 'degree', 'radius', 'color', 'cluster']

In [16]:
min_val = processed_df._get_numeric_data().min().min().round()
max_val = processed_df._get_numeric_data().max().max().round()
processed_df = processed_df[list(correlation_df.identifier) + ['group']]

In [37]:
from IPython.core.display import display, HTML

In [38]:
@interact
def plot_parallel_plot(cluster=correlation_df.cluster.unique()):
    cluster_colors = dict(zip(correlation_df.cluster, correlation_df.color))
    clusters = correlation_df.groupby('cluster')
    identifiers = clusters.get_group(cluster)['identifier'].tolist()
    title= "Parallel plot cluster: {}".format(cluster)
    df = processed_df.set_index('group')[identifiers].reset_index()
    figure = plots.basicFigures.get_parallel_plot(df, identifier=cluster, args={'color':cluster_colors[cluster], 'group':'group', 
                                                                          'title':title,
                                                                          'zscore':False})
    display(HTML("<p>{}</p>".format(",".join(identifiers))))
    iplot(figure.figure)

interactive(children=(Dropdown(description='cluster', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1…